In [1]:
import ollama
import json

# Definimos el modelo 
MODELO = "mistral"

In [2]:
def construir_prompt(texto_cv):
    return f"""
   Eres un asistente experto en datos de RR. HH. Tu tarea es extraer información estructurada del texto del currículum que se proporciona a continuación.
    
    OUTPUT FORMAT:
    Debe generar SOLO un objeto JSON válido. No añada explicaciones, formato Markdown ni texto adicional.
    
    JSON STRUCTURE:
    {{
        "nombre": "Candidate Name",
        "email": "email@example.com",
        "telefono": "phone number",
        "skills": ["skill1", "skill2", "skill3"],
        "experiencia": [
            {{
                "puesto": "Job Title",
                "empresa": "Company Name",
                "periodo": "Start - End",
                "descripcion": "Brief summary of duties"
            }}
        ],
        "educacion": [
            {{
                "titulo": "Degree",
                "institucion": "University",
                "anio": "Year"
            }}
        ]
    }}

    RESUME TEXT:
    {texto_cv}
    """

In [3]:
# Simulacion con pdfplumber 
texto_prueba = """
Juan Pérez
Desarrollador Python Senior
Email: juan.perez@email.com | Tel: +51 999 000 111
Experiencia:
2020-Presente: Tech Lead en Banco BCP. Lideré migración a nube AWS.
2018-2020: Dev Junior en Software SAC. Uso de Django y React.
Educación:
Ingeniería de Sistemas - UNCP (2018)
Skills: Python, Docker, AWS, SQL, English (B2)
"""

print(f"Enviando a {MODELO}.")

try:
    response = ollama.chat(model=MODELO, messages=[
        {
            'role': 'user',
            'content': construir_prompt(texto_prueba),
        },
    ])
    
    resultado_raw = response['message']['content']
    print("\n--- RESPUESTA DEL LLM ---")
    print(resultado_raw)
    
    # Validamos si es JSON real
    datos_estructurados = json.loads(resultado_raw)
    print("\n ¡ÉXITO! JSON válido detectado.")
    print(f"Nombre detectado: {datos_estructurados.get('nombre')}")
    print(f"Skills detectadas: {len(datos_estructurados.get('skills', []))}")

except json.JSONDecodeError:
    print("\n ERROR: El modelo no devolvió un JSON limpio. A veces pasa, hay que ajustar el prompt.")
except Exception as e:
    print(f"\n Error de conexión: {e}")

Enviando a mistral.

--- RESPUESTA DEL LLM ---
 {
        "nombre": "Juan Pérez",
        "email": "juan.perez@email.com",
        "telefono": "+51 999 000 111",
        "skills": ["Python", "Docker", "AWS", "SQL", "English (B2)"],
        "experiencia": [
            {
                "puesto": "Tech Lead",
                "empresa": "Banco BCP",
                "periodo": "2020-Presente",
                "descripcion": "Lideré migración a nube AWS."
            },
            {
                "puesto": "Dev Junior",
                "empresa": "Software SAC",
                "periodo": "2018-2020",
                "descripcion": "Uso de Django y React."
            }
        ],
        "educacion": [
            {
                "titulo": "Ingeniería de Sistemas",
                "institucion": "UNCP",
                "anio": "2018"
            }
        ]
    }

 ¡ÉXITO! JSON válido detectado.
Nombre detectado: Juan Pérez
Skills detectadas: 5
